<a href="https://colab.research.google.com/github/fatemafaria142/Instruction-Guided-Code-Generation-Pipeline/blob/main/Code_Generation_using_GPT_2_Medium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.1 MB/s eta 0:00:00


In [3]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 25.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [4]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.7/270.7 kB 2.0 MB/s eta 0:00:00


In [5]:
!pip install accelerate -U

## **Dataset Link:** https://huggingface.co/datasets/sahil2801/CodeAlpaca-20k

In [6]:
from datasets import load_dataset

dataset = load_dataset("sahil2801/CodeAlpaca-20k")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['input', 'instruction', 'output'],
        num_rows: 20022
    })
})


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Get the first 5000 data points**

In [9]:
# Get the first 5000 data points
num_samples_to_display = 5000
subset_dataset = dataset['train'].select(range(num_samples_to_display))

# Display information for 3 data points from the subset
num_samples_to_show = 5
for i in range(num_samples_to_show):
    data = subset_dataset[i]
    print(f"Data Point {i + 1}:")
    print("Instruction:", data['instruction'])
    print("Input:", data['input'])
    print("Output:", data['output'])
    print("\n-----------------------------\n")

Data Point 1:
Instruction: Create an array of length 5 which contains all even numbers between 1 and 10.
Input: 
Output: arr = [2, 4, 6, 8, 10]

-----------------------------

Data Point 2:
Instruction: Formulate an equation to calculate the height of a triangle given the angle, side lengths and opposite side length.
Input: 
Output: Height of triangle = opposite side length * sin (angle) / side length

-----------------------------

Data Point 3:
Instruction: Write a replace method for a string class which replaces the given string with a given set of characters.
Input: string = "Hello World!"
replace_with = "Greetings!"
Output: def replace(self, replace_with):
    new_string = ""
    for char in self:
        if char == " ":
            new_string += replace_with
        else:
            new_string += char
    return new_string

-----------------------------

Data Point 4:
Instruction: Create an array of length 15 containing numbers divisible by 3 up to 45.
Input: 
Output: arr = [3, 

# **Functions to Generate Prompts from Dataset Entries**

In [10]:
def get_prompt_with_input(x):
    result = f"### Instruction:\n{x['instruction']}\n\n"
    result += f"### Input:\n{x['input']}\n\n"
    result += f"### Response:\n{x['output']}"
    return result

def get_prompt_without_input(x):
    result = f"### Instruction:\n{x['instruction']}\n\n"
    result += f"### Response:\n{x['output']}"
    return result

def get_prompt(x):
    if x['input'] == '':
        return get_prompt_without_input(x)
    else:
        return get_prompt_with_input(x)


# **Display prompts for the first 5 data points**

In [11]:
# Generate prompts for each data point in the subset dataset
prompts = []
for i in range(num_samples_to_display):
    data = subset_dataset[i]
    prompt = get_prompt(data)
    prompts.append(prompt)

# Display the generated prompts or use them as needed
for idx, prompt in enumerate(prompts[:5]):  # Display prompts for the first 5 data points
    print(f"Prompt for Data Point {idx + 1}:")
    print(prompt)
    print("\n-----------------------------\n")


Prompt for Data Point 1:
### Instruction:
Create an array of length 5 which contains all even numbers between 1 and 10.

### Response:
arr = [2, 4, 6, 8, 10]

-----------------------------

Prompt for Data Point 2:
### Instruction:
Formulate an equation to calculate the height of a triangle given the angle, side lengths and opposite side length.

### Response:
Height of triangle = opposite side length * sin (angle) / side length

-----------------------------

Prompt for Data Point 3:
### Instruction:
Write a replace method for a string class which replaces the given string with a given set of characters.

### Input:
string = "Hello World!"
replace_with = "Greetings!"

### Response:
def replace(self, replace_with):
    new_string = ""
    for char in self:
        if char == " ":
            new_string += replace_with
        else:
            new_string += char
    return new_string

-----------------------------

Prompt for Data Point 4:
### Instruction:
Create an array of length 15 

# **More Examples of Prompts**

In [12]:
# Generate prompts for each data point in the subset dataset
prompts = []
for i in range(num_samples_to_display):
    data = subset_dataset[i]
    prompt = get_prompt(data)
    prompts.append(prompt)

# Display the generated prompts or use them as needed
for idx, prompt in enumerate(prompts[5:10]):  # Display prompts for the first 3 data points
    print(f"Prompt for Data Point {idx + 1}:")
    print(prompt)
    print("\n-----------------------------\n")


Prompt for Data Point 1:
### Instruction:
Create a nested loop to print every combination of numbers between 0-9

### Response:
for i in range(10):
    for j in range(10):
        print(i, j)

-----------------------------

Prompt for Data Point 2:
### Instruction:
Write a function to find the maximum difference between two numbers in a given array.

### Input:
arr = [5, 3, 17, 11, 9]

### Response:
def max_difference(arr):
    max_diff = 0
    for i in range(len(arr)-1):
        diff = arr[i+1] - arr[i]
        if diff > max_diff:
            max_diff = diff
    return max_diff

-----------------------------

Prompt for Data Point 3:
### Instruction:
Write a function to generate the nth Fibonacci number.

### Input:
< noinput >

### Response:
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-2) + fib(n-1)

-----------------------------

Prompt for Data Point 4:
### Instruction:
Write a class to represent a 2D point with x and 

# **GPT-2 Medium and its tokenizer**
* https://huggingface.co/openai-community/gpt2-medium

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained('gpt2-medium')
model = AutoModelForCausalLM.from_pretrained('gpt2-medium')

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [14]:
# Set the padding token for the tokenizer
tokenizer.pad_token = tokenizer.eos_token

# **Dataset Preparation and Tokenization for GPT-2 Medium Training**
* This code segment appears to perform the following tasks:

* Assumes a dataset with specific keys ('instruction', 'input', and 'output').
* Iterates through the dataset to generate prompts based on the available data points.
* Tokenizes the generated prompts and output text using a tokenizer, preparing them as tensors.
* Compiles tokenized inputs, labels, and attention masks to be used for GPT-2 model training.

In [15]:
from transformers import Trainer, TrainingArguments
import torch
# Assuming subset_dataset is your dataset with 'instruction', 'input', 'output', and 'text' keys
dataset = subset_dataset  # No need for ['train'] if keys are 'instruction', 'input', 'output', and 'text'

# Initialize empty lists to store inputs, targets, and attention masks
input_ids = []
labels = []
attention_masks = []

# Tokenize and prepare the dataset
for data_point in dataset:
    # Generate prompt
    prompt = get_prompt(data_point)

    # Tokenize prompt
    tokenized_prompts = tokenizer(prompt, return_tensors="pt", truncation=True, padding="max_length", max_length=128)

    # Tokenize output text (corrected text)
    tokenized_output = tokenizer(data_point['output'], return_tensors="pt", truncation=True, padding="max_length", max_length=128)

    # Append tokenized inputs, labels, and attention masks
    input_ids.append(tokenized_prompts['input_ids'])
    labels.append(tokenized_output['input_ids'])
    attention_masks.append(tokenized_prompts['attention_mask'])

# Convert lists to tensors
input_ids = torch.stack(input_ids)
labels = torch.stack(labels)
attention_masks = torch.stack(attention_masks)

### **Printing input_ids, labels, and attention_masks for the first 5 examples**

In [16]:
# Assuming the code provided earlier to prepare the dataset is already executed

# Printing input_ids, labels, and attention_masks for the first 5 examples
for i in range(3):
    print(f"Example {i+1}:")
    print("Input IDs:", input_ids[i])
    print("Labels:", labels[i])
    print("Attention Mask:", attention_masks[i])
    print("-----------------------")


Example 1:
Input IDs: tensor([[21017, 46486,    25,   198, 16447,   281,  7177,   286,  4129,   642,
           543,  4909,   477,   772,  3146,  1022,   352,   290,   838,    13,
           198,   198, 21017, 18261,    25,   198,  3258,   796,   685,    17,
            11,   604,    11,   718,    11,   807,    11,   838,    60, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 

# **Dynamic Data Collation for GPT-2 Medium Model Training**
* This code segment encompasses a class, GPT2DataCollator, which is designed to handle the collation of input features for GPT-2 model training. It dynamically checks the type of input features (whether they are dictionaries or tuples) and appropriately extracts input IDs, attention masks, and labels for padding and preparing the data to the same length. This data collation process is crucial for ensuring the uniformity and compatibility of the input features during the training of the GPT-2 Medium model.

In [17]:
from transformers import Trainer, TrainingArguments
import torch


class GPT2DataCollator:
    def __call__(self, features):
        # Check if the features are dictionaries or tuples
        if isinstance(features[0], dict):
            input_ids = [feature['input_ids'] for feature in features]
            attention_masks = [feature['attention_mask'] for feature in features]
            labels = [feature['labels'] for feature in features]
        else:  # Assuming features are tuples
            input_ids = [feature[0] for feature in features]
            attention_masks = [feature[1] for feature in features]
            labels = [feature[2] for feature in features]

        # Pad inputs and labels to the same length
        input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
        attention_masks = torch.nn.utils.rnn.pad_sequence(attention_masks, batch_first=True, padding_value=0)
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=tokenizer.pad_token_id)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_masks,
            'labels': labels
        }


# **Define the Training Arguments and Trainer**

In [18]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./gpt2-finetuned-version-1',    # Directory to save the model and checkpoints
    num_train_epochs=5,               # Number of training epochs
    per_device_train_batch_size=4,    # Batch size per device during training
    save_steps=500,                  # Save checkpoint every X steps
    logging_dir='./logs',             # Directory for storing logs
    logging_steps=500,                # Log training metrics every X steps
    evaluation_strategy="epoch",      # Evaluation strategy to adopt during training
    report_to="none",                 # Disable evaluation during training
    prediction_loss_only=True,        # Compute only the prediction loss
    warmup_steps=500# number of warmup steps for learning rate scheduler
    # Add any additional arguments as needed
)

# Initialize the Trainer with the custom data collator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=torch.utils.data.TensorDataset(input_ids, attention_masks, labels),
    eval_dataset=torch.utils.data.TensorDataset(input_ids, attention_masks, labels),
    data_collator=GPT2DataCollator()  # Use the custom data collator
)

In [19]:
# Start training
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.618100,2.459111
2,2.422800,2.225909
3,2.223800,2.073506
4,2.085900,1.966073
5,2.000500,1.908532


TrainOutput(global_step=6250, training_loss=2.33378359375, metrics={'train_runtime': 4260.5009, 'train_samples_per_second': 5.868, 'train_steps_per_second': 1.467, 'total_flos': 5804379340800000.0, 'train_loss': 2.33378359375, 'epoch': 5.0})

In [20]:

trainer.save_model()


# **Load the save model and train it again here...**

In [ ]:
'''
from transformers import Trainer, TrainingArguments
# Load the previously trained model
model_path = './gpt2-finetuned-version-1'  # Replace this with the path to your saved model
model = AutoModelForCausalLM.from_pretrained(model_path)

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./gpt2-version-1',    # Directory to save the model and checkpoints
    num_train_epochs=10,               # Number of training epochs
    per_device_train_batch_size=4,    # Batch size per device during training
    save_steps=1000,                  # Save checkpoint every X steps
    logging_dir='./logs',             # Directory for storing logs
    logging_steps=500,                # Log training metrics every X steps
    evaluation_strategy="epoch",      # Evaluation strategy to adopt during training
    report_to="none",                 # Disable evaluation during training
    prediction_loss_only=True,        # Compute only the prediction loss
    warmup_steps=500# number of warmup steps for learning rate scheduler
    # Add any additional arguments as needed
)

# Initialize the Trainer with the custom data collator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=torch.utils.data.TensorDataset(input_ids, attention_masks, labels),
    eval_dataset=torch.utils.data.TensorDataset(input_ids, attention_masks, labels),
    data_collator=GPT2DataCollator()  # Use the custom data collator
)
'''

In [ ]:
'''
# Start training
trainer.train()
'''

Epoch,Training Loss,Validation Loss
1,2.574100,2.396716
2,2.429800,2.298020
3,2.313800,2.165554
4,2.241800,2.081761
5,2.166700,2.018137
6,2.105900,1.974059
7,2.015700,1.922854
8,2.012600,1.887941
9,1.977500,1.865913
10,1.932800,1.855636


TrainOutput(global_step=12500, training_loss=2.1688069775390626, metrics={'train_runtime': 3040.5187, 'train_samples_per_second': 16.445, 'train_steps_per_second': 4.111, 'total_flos': 3266150400000000.0, 'train_loss': 2.1688069775390626, 'epoch': 10.0})

In [ ]:
'''
#Save the fine-tuned model here
trainer.save_model()
'''

In [ ]:
# Evaluate perplexity on the test dataset
eval_result = trainer.evaluate(eval_dataset=torch.utils.data.TensorDataset(input_ids, attention_masks, labels))
print("Perplexity:", eval_result['eval_loss'])

# **Text Generation Using Fine-Tuned GPT-2 Medium Model Pipeline**

In [ ]:
from transformers import pipeline

# Define the generator pipeline using the fine-tuned GPT-2 model and tokenizer
generator = pipeline('text-generation', model='./gpt2-version-1', tokenizer='gpt2', pad_token_id=tokenizer.pad_token_id)

# Test prompt for generating text
test_prompt = (
    "### Instruction:\r\nWrite code that removes spaces from a given string.\r\n\r\n"
    "### Input:\r\nstring = \" A B C D \"\r\n\r\n"
    "### Response:"
)

# Generate text based on the test prompt using the generator pipeline
generated_test_text = generator(test_prompt, max_length=100, num_return_sequences=1)

# Print the generated text for the test prompt
print(generated_test_text[0]['generated_text'])

### Instruction:
Write code that removes spaces from a given string.

### Input:
string = " A B C D "

### Response:		


# **More Text Generation Examples**

In [ ]:
from datasets import load_dataset

dataset = load_dataset("sahil2801/CodeAlpaca-20k")

In [ ]:
# Get the first 10 data points
num_samples_to_display = 10
subset_dataset = dataset['train'].select(range(num_samples_to_display))

# Display information for 3 data points from the subset
num_samples_to_show = 10
for i in range(num_samples_to_show):
    data = subset_dataset[i]
    print(f"Data Point {i + 1}:")
    print("Instruction:", data['instruction'])
    print("Input:", data['input'])
    print("Output:", data['output'])
    print("\n-----------------------------\n")

Data Point 1:
Instruction: Create an array of length 5 which contains all even numbers between 1 and 10.
Input: 
Output: arr = [2, 4, 6, 8, 10]

-----------------------------

Data Point 2:
Instruction: Formulate an equation to calculate the height of a triangle given the angle, side lengths and opposite side length.
Input: 
Output: Height of triangle = opposite side length * sin (angle) / side length

-----------------------------

Data Point 3:
Instruction: Write a replace method for a string class which replaces the given string with a given set of characters.
Input: string = "Hello World!"
replace_with = "Greetings!"
Output: def replace(self, replace_with):
    new_string = ""
    for char in self:
        if char == " ":
            new_string += replace_with
        else:
            new_string += char
    return new_string

-----------------------------

Data Point 4:
Instruction: Create an array of length 15 containing numbers divisible by 3 up to 45.
Input: 
Output: arr = [3, 

In [ ]:
from transformers import pipeline

# Define the generator pipeline using the fine-tuned GPT-2 model and tokenizer
generator = pipeline('text-generation', model='./gpt2-version-1', tokenizer='gpt2', pad_token_id=tokenizer.pad_token_id)

# Generate text based on the subset of the training dataset
for example in subset_dataset:
    test_prompt = (
        "Below is an instruction that describes a task. \r\n\r\n"
        f"### Instruction:\r\n{example['instruction']}\r\n\r\n### Input:\r\n{example['input']}\r\n\r\n### Response:"  # Replace this with your prompt logic
    )

    # Generate text based on the test prompt using the generator pipeline
    generated_text = generator(test_prompt, max_length=100, num_return_sequences=1)

    # Print the generated text for each test prompt
    print(generated_text[0]['generated_text'])
    print("------------------------------------------")


Below is an instruction that describes a task. 

### Instruction:
Create an array of length 5 which contains all even numbers between 1 and 10.

### Input:


### Response:
------------------------------------------
Below is an instruction that describes a task. 

### Instruction:
Formulate an equation to calculate the height of a triangle given the angle, side lengths and opposite side length.

### Input:


### Response:ith
------------------------------------------
Below is an instruction that describes a task. 

### Instruction:
Write a replace method for a string class which replaces the given string with a given set of characters.

### Input:
string = "Hello World!"
replace_with = "Greetings!"

### Response:string: replaced
)
------------------------------------------
Below is an instruction that describes a task. 

### Instruction:
Create an array of length 15 containing numbers divisible by 3 up to 45.

### Input:


### Response: in
------------------------------------------
Belo